In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import date
import re
import tkinter as tk
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.colors as mcolors
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.pyplot as plt
import seaborn as sns
import webbrowser
import os
import warnings
warnings.filterwarnings("ignore")

class App:
    def __init__(self, master):
        self.master = master
        self.master.title("Ingreso de URL")
        
        self.style = ttk.Style()
        self.style.theme_use('clam')

        self.window_width = 400
        self.window_height = 200
        self.screen_width = self.master.winfo_screenwidth()
        self.screen_height = self.master.winfo_screenheight()
        self.position_top = int(self.screen_height / 2 - self.window_height / 2)
        self.position_right = int(self.screen_width / 2 - self.window_width / 2)
        self.master.geometry(f'{self.window_width}x{self.window_height}+{self.position_right}+{self.position_top}')
        
        self.master.attributes('-topmost', 1)

        self.url_input = tk.StringVar() 

        ttk.Label(self.master, text="Por favor, ingrese la URL que desea analizar:").pack(pady=20)
        self.url_entry = ttk.Entry(self.master, width=50, textvariable=self.url_input)
        self.url_entry.pack(pady=10)

        ttk.Button(self.master, text="Comenzar", command=self.on_submit).pack(pady=20)

    def on_submit(self):
        url = self.url_entry.get()
        self.master.withdraw() 
        self.extraer_datos(url)

    def extraer_datos(self, url):
        response = requests.get(url)
        if response.status_code != 200:
            print("Error al obtener la página. Código de estado:", response.status_code)
            return None
        soup = BeautifulSoup(response.content, 'html.parser')

        items = soup.find_all('div', class_='ui-search-result__content-wrapper')

        titulos = []
        urls = []
        precios = []
        cuotas = []

        for item in items:
            estado = item.find('span', class_='ui-search-item__group__element ui-search-item__details')
            if estado and 'Usado' in estado.text:
                continue
            titulo = item.find('h2', class_='ui-search-item__title')
            titulos.append(titulo.text if titulo else 'Sin título')
            url = item.find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')
            urls.append(url['href'] if url else 'Sin URL')
            precio = item.find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript')
            precios.append(precio.text if precio else 'Sin precio')
            cuota_info = item.find('div', class_='ui-search-item__group__element ui-search-installments ui-search-color--LIGHT_GREEN')
            cuotas.append(cuota_info.text.strip() if cuota_info else 'Sin cuotas')

        df = pd.DataFrame({'Publicaciones': titulos, 'URL': urls, 'Precio': precios, 'Cuotas': cuotas})
        df['Fecha'] = date.today()
        
        df['Precio_numerico'] = df['Precio'].apply(self.limpiar_precio)

        self.mostrar_frame_progreso(df)

    def limpiar_precio(self, precio):
        if not precio:
            return np.nan
        if 'US$' in precio:
            precio_limpio = re.sub(r'[^0-9,]', '', precio.replace('US$', ''))
            precio_limpio = precio_limpio.replace(',', '')
            try:
                return float(precio_limpio) * 1300
            except ValueError:
                return np.nan
        else:
            precio_limpio = re.sub(r'[^0-9,]', '', precio)
            precio_limpio = precio_limpio.replace(',', '')
            try:
                return float(precio_limpio)
            except ValueError:
                return np.nan

    def mostrar_frame_progreso(self, df):
        self.progress_window = tk.Toplevel(self.master)
        self.progress_window.title("Proceso de Extracción")
        self.progress_window.geometry('400x200')
        
        frame_progreso = tk.Frame(self.progress_window, width=300, height=150)
        frame_progreso.pack(pady=20)
        lbl_cargando = tk.Label(frame_progreso, text="Extrayendo información...", font=("Arial", 12))
        lbl_cargando.pack(pady=10)
        progress_bar = ttk.Progressbar(frame_progreso, orient=tk.HORIZONTAL, length=300, mode='determinate')
        progress_bar.pack(pady=10)
        progress_bar['maximum'] = 100
        
        boton_finalizar = tk.Button(frame_progreso, text="Finalizar", state='disabled', command=lambda: self.cerrar_progreso(df))
        boton_finalizar.pack(pady=10)

        def realizar_extraccion():
            max_value = len(df['URL'])
            progress_bar['maximum'] = max_value

            stocks = []
            with ThreadPoolExecutor(max_workers=10) as executor:
                futures = {executor.submit(self.extraer_stock_disponible, url): url for url in df['URL']}
                for future in as_completed(futures):
                    result = future.result()
                    stocks.append(result)
                    progress_bar['value'] += 1
                    progress_bar.update_idletasks()

            df['Stock Disponible'] = stocks
            df['Stock Disponible'] = df['Stock Disponible'].apply(lambda x: 'Última Disponible' if x == 0 else x)

            self.master.after(0, lambda: self.habilitar_boton_finalizar(boton_finalizar))

        threading.Thread(target=realizar_extraccion).start()
        
    def cerrar_progreso(self, df_sorted_visible):
        self.guardar_csv(df_sorted_visible)
        self.progress_window.destroy()
        self.mostrar_resultados(df_sorted_visible)

    def habilitar_boton_finalizar(self, boton_finalizar):
        boton_finalizar.config(state='normal')

    def extraer_stock_disponible(self, url):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                stock_info = soup.find('span', class_='ui-pdp-buybox__quantity__available')
                if stock_info:
                    stock_text = stock_info.get_text()
                    stock_number = re.search(r'\d+', stock_text)
                    if stock_number:
                        return int(stock_number.group())
        except requests.RequestException as e:
            print(f"Error al acceder a {url}: {e}")
        return 0

    def mostrar_resultados(self, df):
        df_sorted = df.copy()
        df_sorted['Precio_numerico'] = df_sorted['Precio_numerico'].fillna(0)
        df_sorted = df_sorted.sort_values(by='Precio_numerico', ascending=True)

        self.df_original = df_sorted.copy()

        column_order = ['Publicaciones', 'URL', 'Precio', 'Cuotas', 'Stock Disponible', 'Fecha', 'Precio_numerico']
        df_sorted = df_sorted[column_order]

       # df_sorted_visible = df_sorted.drop(columns=['Precio_numerico'])

        self.mostrar_dataframe(df_sorted)


    def graficar_precios(self, df_sorted):
        enova = df_sorted[df_sorted['Publicaciones'].str.contains('Enova')]
        if not enova.empty:
            precio_enova = enova.iloc[0]['Precio_numerico']
            productos_mas_baratos = df_sorted[df_sorted['Precio_numerico'] <= precio_enova]
            if productos_mas_baratos.empty:
                mensaje_ventana = tk.Toplevel()
                mensaje_ventana.title("Felicitaciones")
                label = tk.Label(mensaje_ventana, text="Tu producto Enova es el más barato... ¡Felicidades!", padx=20, pady=20)
                label.pack()
                button = tk.Button(mensaje_ventana, text="Cerrar", command=mensaje_ventana.destroy)
                button.pack(pady=10)
                mensaje_ventana.mainloop()
            else:
                productos_mas_baratos = productos_mas_baratos.sort_values(by='Precio_numerico', ascending=False)

                def crear_figura():
                    fig, ax = plt.subplots(figsize=(10, 8))

                    norm = plt.Normalize(productos_mas_baratos['Precio_numerico'].min(), productos_mas_baratos['Precio_numerico'].max())
                    sm = plt.cm.ScalarMappable(cmap="Reds", norm=norm)
                    sm.set_array([])
                    cbar = fig.colorbar(sm, ax=ax)
                    cbar.set_label('Precio (en pesos)', rotation=270, labelpad=15)

                    sns.barplot(
                        x=productos_mas_baratos['Publicaciones'], 
                        y=productos_mas_baratos['Precio_numerico'], 
                        palette="Reds_r", 
                        ax=ax
                    )

                    for patch in ax.patches:
                        current_width = patch.get_width()
                        diff = current_width - 0.4
                        patch.set_width(0.4)
                        patch.set_x(patch.get_x() + diff * .5)

                    ax.set_xticklabels(productos_mas_baratos['Publicaciones'], rotation=45, ha='right')
                    ax.set_title(f"Productos más baratos que el Enova ({precio_enova} pesos)")
                    ax.set_xlabel('Productos')
                    ax.set_ylabel('Precio (en pesos)')

                    return fig

                grafico_ventana = tk.Toplevel(self.master)
                grafico_ventana.title("Gráfico de Productos Más Baratos que Enova")

                canvas = FigureCanvasTkAgg(crear_figura(), master=grafico_ventana)
                canvas.draw()
                canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

                def on_resize(event):
                    fig = crear_figura()
                    canvas.figure = fig
                    canvas.draw()
                    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

                grafico_ventana.bind("<Configure>", on_resize)

                button = tk.Button(master=grafico_ventana, text="Cerrar", command=grafico_ventana.destroy)
                button.pack(side=tk.BOTTOM)
                grafico_ventana.mainloop()
        else:
            print("No se encontró ningún producto con 'Enova' en el título.")

    def mostrar_dataframe(self, df_sorted):
        self.resultados_window = tk.Toplevel(self.master)
        self.resultados_window.title("Listado de Productos Ordenados por precio")

        screen_width = self.resultados_window.winfo_screenwidth()
        screen_height = self.resultados_window.winfo_screenheight()
        window_width = 1200 
        window_height = screen_height // 2  
        self.resultados_window.geometry(f'{window_width}x{window_height}')

        frame = tk.Frame(self.resultados_window)
        frame.pack(fill='both', expand=True)

        treeview = ttk.Treeview(frame, columns=list(df_sorted.columns), show='headings')

        for column in df_sorted.columns:
            treeview.heading(column, text=column)
            treeview.column(column, anchor='center')

        for index, row in df_sorted_visible.iterrows():
            if 'Enova' in row['Publicaciones']:
                treeview.insert("", "end", values=list(row), tags=('enova',))
            else:
                treeview.insert("", "end", values=list(row))

        treeview.tag_configure('enova', background='lightgreen')

        scrollbar = ttk.Scrollbar(frame, orient="vertical", command=treeview.yview, style="Vertical.TScrollbar")
        style = ttk.Style()
        style.theme_use('clam')
        style.configure("Vertical.TScrollbar", troughcolor="gray", gripcount=0, borderwidth=0)
        scrollbar.pack(side="right", fill="y")

        treeview.configure(yscrollcommand=scrollbar.set)

        treeview.pack(fill='both', expand=True)
        treeview.bind("<Button-1>", self.on_click)

        button_frame = tk.Frame(self.resultados_window)
        button_frame.pack(pady=10)

        button_cerrar = tk.Button(button_frame, text="Cerrar", command=self.resultados_window.destroy, width=10)
        button_cerrar.pack(side=tk.LEFT, padx=5)
        
        button_graficar = tk.Button(button_frame, text="Graficar precios", command=lambda: self.graficar_precios(self.df_sorted), width=15)
        button_graficar.pack(side=tk.LEFT, padx=5)


    def on_click(self, event):
        treeview = event.widget
        item = treeview.identify_row(event.y)
        column = treeview.identify_column(event.x)
        column_index = int(column.replace('#', '')) - 1 

        if item:
            if column_index == 1:
                url = treeview.item(item)['values'][column_index]
                if url and url != 'Sin URL':
                    webbrowser.open(url)

    def obtener_nombre_archivo_consecutivo(self, base_name, extension):
        files = os.listdir('.')
        
        pattern = re.compile(rf'{re.escape(base_name)}(\d*)\.{re.escape(extension)}')
        matching_files = [f for f in files if pattern.match(f)]
        
        numbers = []
        for file in matching_files:
            match = pattern.match(file)
            if match.group(1):
                numbers.append(int(match.group(1)))
            else:
                numbers.append(0)
        
        if numbers:
            next_number = max(numbers) + 1
        else:
            next_number = 1
        
        new_filename = f"{base_name}{next_number}.{extension}"
        return new_filename
    
    def guardar_csv(self, df_sorted_visible):
        base_name = 'productos_ordenados'
        extension = 'csv'
        df_sorted_visible = df_sorted_visible.sort_values(by='Precio_numerico') 
        csv_filename = self.obtener_nombre_archivo_consecutivo(base_name, extension)
        df_sorted_visible.to_csv(csv_filename, index=False)
        print(f"DataFrame guardado en {csv_filename}")
        os.startfile(csv_filename)


def main():
    root = tk.Tk()
    app = App(root)
    root.mainloop()

if __name__ == "__main__":
    main()
